# Look to bottom for QA Section. 

### Usual error calculation code:

In [1]:
import numpy as np
from typing import Iterable, Tuple, Union
import pandas as pd
from IPython.display import Markdown

# Define our rules...

def rule_1(c, error_val):
    return abs(c) * error_val

def rule_2(c, val, error_val, power):
    return abs(c * power * val ** (power - 1)) * error_val

def rule_3(*err_vals: Iterable[float]) -> float:
    """
    Calculate rule 3 from the paper 'Treatment of Data'.
    
    @param err_vals: A list of parameters, being the errors in each value. It is assumed they were summed 
                     together to get the final value.
    
    @returns: The error of all of the sum of the values...
    """
    total = 0
    
    for err in err_vals:
        total += err ** 2
    
    return np.sqrt(total)


def rule_4(value: float, *error_list: Iterable[Tuple[float, float, float]]) -> float:
    """
    Calculate rule 4 from the paper 'Treatment of Data'.
    
    @param value: The value of the thing we are trying to calculate the error of.
    @param error_list: A list of length 3 tuples. 
                       Each tuple should contain:
                       - A float: A value in the error formula.
                       - A float: The measured error in the above value.
                       - A float: The power of the above value in the multiplicative formula.

    @returns: A float, being the error in 'value'.
    """
    total = 0
    
    for x, x_err, power in error_list:
        total += (power * (x_err / x)) ** 2
        
    return abs(value) * np.sqrt(total)

# Fixed bug in checking code below...
FloatVec = Union[float, np.ndarray] 
BoolVec = Union[bool, np.ndarray]

def values_agree(val_1: FloatVec, err_1: FloatVec, val_2: FloatVec, err_2: FloatVec) -> BoolVec:
    """
    Determines if the values in 2 vectors(arrays) agree with each other, given there uncertainty values. 
    
    @param val_1: The 1st array of values.
    @param err_1: The uncertainty values for the 1st vector.
    @param val_2: The 2nd array of values.
    @param err_2: The uncertainty values for the 2st vector. 
    
    @returns: A vector of booleans, being whether each value agrees with the other.
    """
    # Grab the ranges for each value...
    r11, r12 = val_1 - err_1, val_1 + err_1
    r21, r22 = val_2 - err_2, val_2 + err_2
    
    # The ranges are sorted (r21 <= r22 and r11 <= r12), so the simple 2 checks below are enough. 
    # Check 1: Does r21(lowest value of 2nd range) land above the 1st range? If so fail...
    # Check 2: Does r22(highest value of 2nd range) fall below the 1st range? If so fail...
    return ((r21 <= r12) & (r11 <= r22))

# Questions:

### 1) Write your measurement of the mass of the aluminum cylinder including the uncertainty.   (3 points)

This is the value I recored for the mass of the aluminum cylinder:

In [53]:
df = pd.read_csv("data.csv")
display(Markdown(fr"Mass of aluminium cylinder: ${df['m (g)'][0]:.02f} \pm {df['err_m (g)'][0]:.02f}\text{{ g}}$."))

Mass of aluminium cylinder: $202.90 \pm 0.10\text{ g}$.

### 2) Write your measurement for the diameter of the cylinder including the uncertainty.  (3 points)

This is the value I recorded for the diameter of the cylinder:

In [52]:
df = pd.read_csv("data.csv")
display(Markdown(fr"Diameter of cylinder: ${df['D (mm)'][0]:.02f} \pm {df['D_err (mm)'][0]:.02f}\text{{ mm}}$."))

Diameter of cylinder: $48.00 \pm 0.50\text{ mm}$.

### 3) What is your uncertainty in $T_{final}$ or $T_{initial}$ and how did you determine it?  (5 points)

My uncertainty for $T_{final}$ and $T_{initial}$ ended up being $0.3 \text{ }^{\circ}C$. I determined from our prior experiment, and the value correlates to the 'bounce' in the thermistor as it levels out and correctly gives the temperature. Sadly, I couldn't determine a value myself due to not having access to the software for converting thermistor resistance values into temperatures.

### 4) What is the mechanical equivalent of heat you calculated including the uncertainty.  Show your work.  (either work done by hand or give me a few lines of the code you used, or an explanation of them) (6 points)

The following code below computes the mechanical equivalent of heat conversion factor. At the bottom it prints out its result:

In [57]:
import pandas as pd
from IPython.display import Markdown

# Read in data I took.
df = pd.read_csv("data.csv")

# Grab values from csv....

# Mass of the hanging object
M = df["M (kg)"]
err_M = df["err_M (kg)"]

# Radius of the cylinder.
R = (df["D (mm)"] / 1000) / 2 # mm -> m
err_R = (df["D_err (mm)"] / 1000) / 2 # mm -> m

# The number of turns completed... 
# (We subtract off turns before cylinder started warming up, and include and error of 10 turns.)
N = df["#Turns"] - df["Sub_Turns"]
err_N = df["err_#Turns"]

# g!!!
g = 9.8  # m / s^2

# Calculate work and error in work done.
Work = 2 * np.pi * g * R * N * M
err_Work = rule_4(Work, (R, err_R, 1), (N, err_N, 1), (M, err_M, 1))

# The change in temperature...
T_diff = df["T_finish (C)"] - df["T_start (C)"]
err_T_diff = rule_3(df["err_T (C)"], df["err_T (C)"])

# Specfic Heat of aluminum...
c_aluminum = 214  # cal/(kg*C) We assume no error.

# Mass of the cylinder...
m = df["m (g)"] / 1000 # g -> kg
err_m = df["err_m (g)"] / 1000 # g -> kg

# Calculate heat and error in head generated.
Heat = T_diff * c_aluminum * m
err_Heat = rule_4(Heat, (m, err_m, 1), (T_diff, err_T_diff, 1))

# Calculate the conversion constant...
J = Work / Heat
err_J = rule_4(J, (Work, err_Work, 1), (Heat, err_Heat, -1))

accepted_J = 4.1860

display(Markdown(fr"Conversion rate for work to heat derived experimentally: ${J[0]:.04f} ± {err_J[0]:.04f} \frac{{J}}{{cal}}$"))
display(Markdown(fr"Accepted value: ${accepted_J:.04f} \frac{{J}}{{cal}}$."))
display(Markdown(f"Do values aggree? {'Yes' if(values_agree(J[0], err_J[0], accepted_J, 0)) else 'No'}."))

Conversion rate for work to heat derived experimentally: $4.3602 ± 0.1797 \frac{J}{cal}$

Accepted value: $4.1860 \frac{J}{cal}$.

Do values aggree? Yes.

As seen above, we get a result of $4.3602 \pm 0.1797 \frac{J}{cal}$.

### 5) Does your result for the mechanical equivalent of heat agree with the accepted value of $J = 4.1860 \frac{J}{cal}$?  Justify your answer.  (4 points)

As seen in the results printed from the code above, my value does agree with the accepted value of $4.1860 \frac{J}{cal}$. This can be easily confirmed by hand by adding and subracting the error from the value we got, and then checking to see if the accepted value lands within the range between the lower and upper error bounds.

### 6) If we had been doing work on a steel cylinder, with specific heat $c_S = 120 \frac{cal}{(kg °C)}$, instead of aluminum, would it have taken more or less turns to achieve the same temperature difference?  Justify your answer.  (4 points)

Rearranging the formula above for work:

$$
J = \frac{W}{Q} \implies W = JQ \implies W = J(mc_s \Delta T)
$$

We see that work and heat are proportional to each other by the constant $J$ ($4.1860 \frac{J}{cal}$). Since work is directly proportional to heat, which is directly proportional to the specific heat of a material, decreasing the specific heat would cause the amount of work we need to do to go down. Therefore, if we had used a steel cylinder, we would have been able to achieve the same temperature difference with less turns (work is proportional to turns).